In [59]:
import pandas as pd
import numpy as np
from PIL import Image

In [3]:
data_path = 'data/full_data/'
presence_only_path = data_path+'Presence_only_occurrences/Presences_only_train_sampled_100.csv'
presence_absence_path = data_path+'Presence_Absence_surveys/Presences_Absences_train.csv'

In [4]:
presence_only_df = pd.read_csv(presence_only_path, sep=";", header='infer', low_memory=False)
presence_only_df.shape

(410899, 16)

In [5]:
presence_absence_df = pd.read_csv(presence_absence_path, sep=";", header='infer', low_memory=False)
presence_absence_df.shape

(85325, 15)

In [13]:
presence_only_df

,Unnamed: 0,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
0,174,401885,3.044675e+09,jenniferkalt,iNaturalist RG,2019-05-24,144,2019,-10.189970,52.18669,2960607.0,3421265.0,7.00,8951,3019043,1174
1,224,4710849,1.990449e+09,jerry2018,iNaturalist RG,2018-08-09,221,2018,-10.133990,51.74629,2950833.0,3372799.0,12.00,763,3019093,1224
2,337,3216493,3.724848e+09,NaN,Observation.org,2018-07-02,183,2018,-10.045470,53.43000,3008133.0,3552399.0,25.00,4834,3019203,1334
3,428,2722297,3.725957e+09,NaN,Observation.org,2018-07-04,185,2018,-9.993714,53.39348,3010299.0,3547527.0,10.00,7294,3019293,1424
4,480,2013160,2.005338e+09,Andrew Lamb,iNaturalist RG,2019-02-14,45,2019,-9.968167,52.07664,2971775.0,3405267.0,6.00,7288,3019345,1476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410894,1792334,4679980,2.644838e+09,NaN,Pl@ntNet automatic,2019-04-30,120,2019,5.869271,45.64443,3998708.0,2512477.0,4.90,10039,4731159,1708029
410895,1358840,1082630,2.235344e+09,NaN,Waarnemingen.be,2017-11-06,310,2017,4.305170,51.33690,3924600.0,3151684.0,10.00,10039,4323236,1301523
410896,630012,415494,3.949691e+09,NaN,Pl@ntNet observations,2021-11-29,333,2021,-0.595808,47.45600,3524573.0,2761853.0,11.86,10039,3613216,595573
410897,1495889,3900230,3.949650e+09,NaN,Pl@ntNet observations,2021-08-30,242,2021,4.769210,52.38190,3965175.0,3265286.0,1.40,10039,4457741,1436096


In [112]:
sites = presence_only_df[['patchID','dayOfYear']].drop_duplicates().reset_index(drop=True)
sites

,patchID,dayOfYear
0,3019043,144
1,3019093,221
2,3019203,183
3,3019293,185
4,3019345,45
...,...,...
392903,4731159,120
392904,4323236,310
392905,3613216,333
392906,4457741,242


In [122]:
species = presence_only_df['speciesId'].sort_values().unique()
species

array([    1,     2,     3, ..., 10037, 10038, 10039])

In [123]:
index = 16
item = sites.iloc[index].to_dict()
item

{'patchID': 3020159, 'dayOfYear': 2}

In [127]:
item_species = presence_only_df[
    (presence_only_df['patchID'] == item['patchID']) & (presence_only_df['dayOfYear'] == item['dayOfYear'])
].speciesId.values
print(item_species)
labels = 1 * np.isin(species, item_species)
print(labels.sum(), 'species')

[ 317 4263 1526 5020 7950]
5 species


In [100]:
size = 20

In [108]:
def crop_img(img, size):
    xmin=round((img.shape[0] - size) /2)
    xmax=round((img.shape[0] + size) /2)
    ymin=round((img.shape[1] - size) /2)
    ymax=round((img.shape[1] + size) /2)
    img = img[xmin:xmax, ymin:ymax]
    return img

In [128]:
list_tensor = {'order': [], 'tensors':[]}

#patchID=3018575 is "./rgb/75/85/3018575.jpeg". I
id_ = str(int(item['patchID']))

rgb_path = f"data/full_data/SatelliteImages/rgb/{id_[-2:]}/{id_[-4:-2]}/{id_}.jpeg"
img = np.asarray(Image.open(rgb_path))

for i, channel in enumerate(['red','green','blue']):
    channel_img = img[:,:,i]
    if size < 128: 
        channel_img = crop_img(channel_img, size)
    print(channel_img.shape)
    list_tensor['tensors'].append(np.expand_dims(channel_img, axis=0))
    list_tensor['order'].append(channel)

nir_path = f"data/full_data/SatelliteImages/nir/{id_[-2:]}/{id_[-4:-2]}/{id_}.jpeg"
nir_img = np.array(Image.open(nir_path))
if size < 128: 
    nir_img = crop_img(nir_img, size)
list_tensor['tensors'].append(np.expand_dims(nir_img,axis=0))
list_tensor['order'].append('nir')
tensor = np.concatenate(list_tensor['tensors'])
tensor.shape
    

(20, 20)
(20, 20)
(20, 20)


(4, 20, 20)